# 🔬 Ontology TTL Generator

OpenAI Assistant API를 활용하여 문서를 분석하고 OWL/RDF 표준에 맞는 TTL 파일을 생성합니다.

## 📋 기능
- 파일 업로드 및 분석
- 목차 기반 문서 구조 분석
- OWL/RDF 표준에 맞는 TTL 파일 생성
- 터미널 메시지로 진행 상황 확인


## 1. 필요한 라이브러리 설치 및 임포트


In [14]:
%pip install --upgrade openai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import time
import re

# 환경 변수 로드
load_dotenv()

print("Successfully import library")


Successfully import library


## 2. 유틸리티 함수들


In [17]:
def print_status(message, status="INFO"):
    """
    터미널에 출력되는 상태메세지 커스텀
    """
    status_colors = {
        "INFO": "\033[94m",      # 파란색
        "SUCCESS": "\033[92m",   # 초록색
        "WARNING": "\033[93m",   # 노란색
        "ERROR": "\033[91m",     # 빨간색
        "RESET": "\033[0m"       # 리셋
    }
    
    color = status_colors.get(status, status_colors["INFO"])
    reset = status_colors["RESET"]
    
    print(f"{color}[{status}] {message}{reset}")


In [10]:
def check_environment():
    """
    환경 변수 설정 확인
    """
    print_status("환경 변수 설정을 확인합니다...", "INFO")
    
    api_key = os.getenv('OPENAI_API_KEY')
    assistant_id = os.getenv('ASSISTANT_ID')
    vector_store_id = os.getenv('VECTOR_STORE_ID')
    
    if not api_key:
        print_status("OpenAI API 키가 설정되지 않았습니다.", "ERROR")
        return False
    
    if not assistant_id:
        print_status("Assistant ID가 설정되지 않았습니다.", "ERROR")
        return False
        
    if not vector_store_id:
        print_status("Vector Store ID가 설정되지 않았습니다.", "ERROR")
        return False
    
    print_status("모든 환경 변수가 올바르게 설정되었습니다.", "SUCCESS")
    return True


In [18]:
def initialize_openai_client():
    """
    OpenAI 클라이언트 초기화
    """
    try:
        client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
        print_status("OpenAI 클라이언트가 성공적으로 초기화되었습니다.", "SUCCESS")
        return client
    except Exception as e:
        print_status(f"OpenAI 클라이언트 초기화 실패: {str(e)}", "ERROR")
        return None

## 3. 메인 처리 함수

In [19]:
def process_files_simple(file_paths, prompt=None):
    """
    파일들을 처리하여 TTL 파일 생성
    """
    print_status("=== Ontology TTL Generator 시작 ===", "INFO")
    
    # 환경 변수 확인
    if not check_environment():
        return False
    
    # OpenAI 클라이언트 초기화
    client = initialize_openai_client()
    if not client:
        return False
    
    # 기본 프롬프트 설정
    if not prompt:
        prompt = """첨부된 파일들을 분석하여 OWL/RDF 표준에 맞는 TTL 파일을 생성해주세요.

                    요구사항:
                    1. 목차를 기준으로 다른 문서들을 분석하세요
                    2. OWL/RDF 표준에 맞는 Turtle 형식으로 작성하세요
                    3. 적절한 네임스페이스, 클래스, 속성을 정의하세요
                    4. 위기경보 수준별 조치사항의 구조를 반영하세요

                    생성할 TTL 파일의 구조:
                    - 기본 네임스페이스 정의
                    - 위기경보 수준 클래스 (관심, 주의, 경계, 심각)
                    - 각 수준별 상황, 조치목록, 조치내용 클래스
                    - 부서별 임무와 역할 클래스
                    - 적절한 속성과 관계 정의"""
    
    print_status(f"프롬프트: {prompt[:100]}...", "INFO")
    
    # 파일 업로드
    uploaded_file_ids = []
    for file_path in file_paths:
        if os.path.exists(file_path):
            try:
                print_status(f"파일 업로드 중: {file_path}", "INFO")
                
                with open(file_path, 'rb') as file:
                    uploaded_file = client.files.create(
                        file=file,
                        purpose='assistants'
                    )
                
                print_status(f"파일 업로드 완료: {uploaded_file.id}", "SUCCESS")
                
                # Vector Store에 파일 추가
                vector_store_id = os.getenv('VECTOR_STORE_ID')
                client.vector_stores.files.create(
                    vector_store_id=vector_store_id,
                    file_id=uploaded_file.id
                )
                
                print_status(f"Vector Store에 파일 추가 완료: {uploaded_file.id}", "SUCCESS")
                uploaded_file_ids.append(uploaded_file.id)
                
            except Exception as e:
                print_status(f"파일 업로드 중 오류 발생: {str(e)}", "ERROR")
        else:
            print_status(f"파일을 찾을 수 없습니다: {file_path}", "ERROR")
    
    if not uploaded_file_ids:
        print_status("업로드된 파일이 없습니다.", "ERROR")
        return False
    
    print_status(f"{len(uploaded_file_ids)}개 파일이 성공적으로 업로드되었습니다.", "SUCCESS")
    
    # Thread 생성 및 실행
    try:
        print_status("Thread를 생성합니다...", "INFO")
        thread = client.threads.create()
        print_status(f"Thread 생성 완료: {thread.id}", "SUCCESS")
        
        # 메시지 추가
        message_content = prompt + "\n\n첨부된 파일들을 분석하여 TTL 파일을 생성해주세요."
        print_status("메시지를 Thread에 추가합니다...", "INFO")
        
        message = client.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=message_content,
            attachments=[{"file_id": file_id, "tools": [{"type": "file_search"}]} 
                       for file_id in uploaded_file_ids]
        )
        
        print_status("메시지 추가 완료", "SUCCESS")
        
        # Assistant 실행
        assistant_id = os.getenv('ASSISTANT_ID')
        print_status("Assistant를 실행합니다...", "INFO")
        
        run = client.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant_id,
            additional_instructions="목차를 기준으로 다른 문서들을 분석하여 OWL/RDF 표준에 맞는 TTL 파일을 생성해주세요. TTL 파일은 Turtle 형식으로 작성하고, 적절한 네임스페이스와 클래스, 속성을 정의해주세요."
        )
        
        print_status(f"Assistant 실행 시작: {run.id}", "SUCCESS")
        
        # 실행 완료 대기
        print_status("Assistant 실행 완료를 기다립니다...", "INFO")
        while True:
            run_status = client.threads.runs.retrieve(
                thread_id=thread.id,
                run_id=run.id
            )
            
            if run_status.status == 'completed':
                print_status("Assistant 실행이 완료되었습니다.", "SUCCESS")
                break
            elif run_status.status == 'failed':
                print_status("Assistant 실행이 실패했습니다.", "ERROR")
                return False
            elif run_status.status == 'requires_action':
                print_status("Assistant가 추가 액션을 요구합니다.", "WARNING")
                return False
            elif run_status.status == 'in_progress':
                print_status("Assistant가 실행 중입니다...", "INFO")
            
            time.sleep(2)
        
        # 응답 가져오기
        print_status("Assistant의 응답을 가져옵니다...", "INFO")
        messages = client.threads.messages.list(
            thread_id=thread.id,
            order="desc"
        )
        
        if messages.data:
            response = messages.data[0].content[0].text.value
            print_status("응답을 성공적으로 가져왔습니다.", "SUCCESS")
            
            # TTL 내용 추출
            print_status("TTL 내용을 추출합니다...", "INFO")
            ttl_patterns = [
                r'```ttl\n(.*?)\n```',
                r'```turtle\n(.*?)\n```',
                r'```\n(.*?)\n```'
            ]
            
            ttl_content = None
            for pattern in ttl_patterns:
                match = re.search(pattern, response, re.DOTALL)
                if match:
                    ttl_content = match.group(1).strip()
                    break
            
            if not ttl_content:
                ttl_content = response.strip()
                print_status("TTL 블록을 찾을 수 없어 전체 응답을 사용합니다.", "WARNING")
            else:
                print_status("TTL 내용을 성공적으로 추출했습니다.", "SUCCESS")
            
            # TTL 파일 저장
            try:
                with open("ontology.ttl", 'w', encoding='utf-8') as f:
                    f.write(ttl_content)
                print_status("TTL 파일이 저장되었습니다: ontology.ttl", "SUCCESS")
                
                print_status("=== TTL 파일 생성 완료 ===", "SUCCESS")
                print_status("생성된 TTL 내용:", "INFO")
                print("=" * 50)
                print(ttl_content)
                print("=" * 50)
                return True
                
            except Exception as e:
                print_status(f"TTL 파일 저장 중 오류 발생: {str(e)}", "ERROR")
                return False
        else:
            print_status("응답을 가져올 수 없습니다.", "ERROR")
            return False
            
    except Exception as e:
        print_status(f"Thread 생성 및 실행 중 오류 발생: {str(e)}", "ERROR")
        return False


In [8]:
# 로컬에 있는 3장 pdf 파일과 목차 경로 정의
from pathlib import Path

chat3_index_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\목차.txt")
actions_level_docs_path = Path(r"C:\Users\user\Desktop\Project\Ontology\위기경보수준별조치사항.pdf")
docs_path = [chat3_index_docs_path, actions_level_docs_path]

In [13]:
process_files_simple(docs_path)

[INFO] === Ontology TTL Generator 시작 ===
[INFO] 환경 변수 설정을 확인합니다...
[SUCCESS] 모든 환경 변수가 올바르게 설정되었습니다.
[SUCCESS] OpenAI 클라이언트가 성공적으로 초기화되었습니다.
[INFO] 프롬프트: 첨부된 파일들을 분석하여 OWL/RDF 표준에 맞는 TTL 파일을 생성해주세요.

                    요구사항:
                    1. 목차를 기...
[INFO] 파일 업로드 중: C:\Users\user\Desktop\Project\Ontology\목차.txt
[SUCCESS] 파일 업로드 완료: file-T7cikFSYTXQoxjqUSduMAb
[SUCCESS] Vector Store에 파일 추가 완료: file-T7cikFSYTXQoxjqUSduMAb
[INFO] 파일 업로드 중: C:\Users\user\Desktop\Project\Ontology\위기경보수준별조치사항.pdf
[SUCCESS] 파일 업로드 완료: file-C8eFEjWub2yDEbBGu3joCq
[SUCCESS] Vector Store에 파일 추가 완료: file-C8eFEjWub2yDEbBGu3joCq
[SUCCESS] 2개 파일이 성공적으로 업로드되었습니다.
[INFO] Thread를 생성합니다...
[ERROR] Thread 생성 및 실행 중 오류 발생: 'OpenAI' object has no attribute 'threads'


False